# Machine Learning Pipeline - Model Training - lightGBM

En este notebook, recogemos los conjuntos de datos transformados y las variables seleccionadas que guardamos en los notebooks anteriores

# Reproducibilidad: sembramos la semilla

Con el objetivo de garantizar la reproducibilidad entre ejecuciones del mismo notebook, pero también entre el entorno de investigación y producción, para cada paso que incluya algún elemento de aleatoriedad, es extremadamente importante que **establezcamos la semilla**

In [18]:
# Para manipulacion de datos
import pandas as pd
import numpy as np

# Para graficar
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.plotting import plot_decision_regions

# Para guardar el modelo
import joblib

# Para construir los modelos
#Clasificación lightGBM (esta en fase experimental)
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

# Para evaluar los modelos
from sklearn.metrics import roc_auc_score, classification_report

# Para visualizar todas las columnas del dataframe
pd.pandas.set_option('display.max_columns', None)

In [19]:
# cargar el conjunto de entrenamiento y el de prueba con las variables diseñadas 
# construimos y guardamos estos conjuntos de datos en el notebook anterior. 

X_train = pd.read_csv('xtrain.csv')
X_test = pd.read_csv('xtest.csv')

X_train.head()

,gender,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,tenure_disc_corr,TotalCharges_disc_corr
0,1,0,0,1,1,0,1,1,1,1,0,1,0,1,0,2,2,2
1,1,0,0,0,1,0,0,2,2,2,2,2,2,2,0,1,3,1
2,0,0,1,0,1,1,2,0,0,1,1,0,0,0,1,1,2,2
3,1,0,1,0,1,1,2,0,1,0,1,0,0,0,1,3,0,0
4,0,0,1,0,1,1,1,1,1,1,0,0,0,2,0,2,3,3


In [20]:
# Cargamos la target

y_train = pd.read_csv('ytrain.csv')
y_test = pd.read_csv('ytest.csv')

y_train.head()

,Churn
0,0
1,0
2,0
3,1
4,0


### Instanciamos el modelo de Arbol de decision: 

Recordamos sembrar la semilla.

In [21]:
# configurar el modelo 
# recuerdar establecer random_state / seed

#Instanciar
hgb = HistGradientBoostingClassifier()

hgb.fit(X_train, y_train)

C:\Users\prose\miniconda3\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


HistGradientBoostingClassifier()

In [22]:
# evalua el modelomos: 

pred = hgb.predict_proba(X_train)[:,1]


print('train auc: {}'.format(
    roc_auc_score(y_train, pred)))
print()

# make predictions for test set
pred = hgb.predict_proba(X_test)[:,1]

# determine auc

print('test auc: {}'.format(
    roc_auc_score(y_test, pred)))
print()

hgb_auc={'Auc':np.round(roc_auc_score(y_test, pred),2)}

train auc: 0.9163852125268978

test auc: 0.840012658554858



In [23]:
hgb.get_params()

{'categorical_features': None,
 'early_stopping': 'auto',
 'l2_regularization': 0.0,
 'learning_rate': 0.1,
 'loss': 'auto',
 'max_bins': 255,
 'max_depth': None,
 'max_iter': 100,
 'max_leaf_nodes': 31,
 'min_samples_leaf': 20,
 'monotonic_cst': None,
 'n_iter_no_change': 10,
 'random_state': None,
 'scoring': 'loss',
 'tol': 1e-07,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

A dia de hoy, este algoritmo no tiene implementada una funcion para la importancia de las variables.

In [24]:
pd.DataFrame(hgb_auc,index=['lightGBM']).to_csv('lightGBM_auc.csv',index=False)